In [2]:
pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19877 sha256=be7984e7f811398b992164d2f69b6c652d64b448b52680d2e962ee91beb6a5b3
  Stored in directory: c:\users\lenovo pro\appdata\local\pip\cache\wheels\7e\ee\d6\47cb94a403bc544de1433986e5530d6b0498021098fbe43aa1
Successfully built vecstack
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U imbalanced-learn

  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
Note: you may need to restart the kernel to use updated packages.


In [150]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import LinearSVC 
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")

In [45]:
trainfile = r'C:/Users/LENOVO PRO/ASU/CIS 508/Assign_03/RevisedHomesiteTrain1.csv'
train_data = pd.read_csv(trainfile)

#train_data = pd.read_csv("C:/Users/admin/Downloads/NewHomesiteTrain.csv")


testfile = r'C:/Users/LENOVO PRO/ASU/CIS 508/Assign_03/RevisedHomesiteTest1.csv'
test_data = pd.read_csv(testfile)

#test_data = pd.read_csv("C:/Users/admin/Downloads/NewHomesiteTest.csv")


print(train_data.shape)
print(test_data.shape)
print(train_data.head())  

(65000, 596)
(173836, 596)
   CoverageField11A  CoverageField11B  CoverageField1A  CoverageField1B  \
0                 2                 1               17               23   
1                 5                 9                6                8   
2                 4                 6                7               12   
3                15                23                3                2   
4                 4                 6                8               13   

   CoverageField2A  CoverageField2B  CoverageField3A  CoverageField3B  \
0               17               23               15               22   
1                6                8                5                7   
2                7               12                6               10   
3                3                2                2                2   
4                8               13                7               11   

   CoverageField4A  CoverageField4B  ...  PropertyField38_N  \
0               16  

In [174]:
y1_train=train_data["QuoteConversion_Flag"]
X1_train=train_data.iloc[:, :-1].copy()
X1_test=test_data.iloc[:, :-1].copy()
print(y1_train.shape)
print(X1_train.shape)
print(X1_test.shape)

print(train_data['QuoteConversion_Flag'].value_counts())

(65000,)
(65000, 595)
(173836, 595)
0    52738
1    12262
Name: QuoteConversion_Flag, dtype: int64


In [184]:
X_train, X_test, y_train, y_test = train_test_split (X1_train, y1_train, test_size=0.2, random_state=1)
X_train, X_test, y_train, y_test

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(52000, 595)
(52000,)
(13000, 595)
(13000,)


In [185]:
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y1_train))
sm = SMOTE(sampling_strategy=1)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))
print(X_res.shape)
print(y_res.shape)

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 42133, 1: 42133})
(84266, 595)
(84266,)


In [180]:
#Construct Decision Tree Model 

clf = DecisionTreeClassifier(min_samples_split=90, max_depth=9)
clf.fit(X_res, y_res)
clf_predict=clf.predict(X_test)
print("accuracy Score (training) for Decision Tree:{0:6f}".format(clf.score(X_train, y_train)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test,clf_predict))

print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(y_test, clf_predict))
clf_cv_score = cross_val_score(clf, X_train,y_train, cv=5, scoring="balanced_accuracy")
print(clf_cv_score)

accuracy Score (training) for Decision Tree:0.896712
Confusion Matrix for Decision Tree
[[9913  642]
 [ 676 1769]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

           0       0.94      0.94      0.94     10555
           1       0.73      0.72      0.73      2445

    accuracy                           0.90     13000
   macro avg       0.83      0.83      0.83     13000
weighted avg       0.90      0.90      0.90     13000

[0.7856183  0.7939771  0.79367961 0.79282478 0.79712444]


In [216]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for Decision Tree:{0:6f}".format(clf.score(X_test1,y_test)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))

#get cross-validation report
clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

{'min_samples_split': 50, 'max_depth': 9}


NameError: name 'X_test1' is not defined

In [ ]:
#Construct Random Forest Model

rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)
print("accuracy Score (training) for RandomForest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))

In [ ]:
#Hyperparameter tuning for random forest classifier
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train1, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

#Construct Random Forest with best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train1,y_train)
rfc_predict = rfc.predict(X_test1)

print("accuracy Score (training) after hypertuning for Random Forest:{0:6f}".format(rfc.score(X_test1,y_test)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))

#get cross-validation report
rfc_cv_score = cross_val_score(rfc, X_train1, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())

In [87]:
#Construct MultiLayer Perceptron Model

mlp = MLPClassifier(max_iter=100)
mlp.fit(X1_train, y1_train)
mlp_predict=mlp.predict(X1_test)
print("accuracy Score (training) for MultiLayer Perceptron:{0:6f}".format(mlp.score(X_test,y_test)))
print("Confusion Matrix for MultiLayer Perceptron:")
print(confusion_matrix(y_test,mlp_predict))

In [ ]:
#Hyperparameter tuning done for MultiLayer Perceptron classifier

#parameters = {'hidden_layer_sizes':[(10,), (20,)], 'activation':['tanh', 'relu'], 'solver':['sgd', 'adam'], 'alpha': [0.0001, 0.05], 'learning_rate':['constant', 'adaptive']}
#parameters = {'hidden_layer_sizes':[(10,5), (20,5)], 'activation':['tanh', 'relu'], 'learning_rate':['constant', 'adaptive']}
parameters = {'hidden_layer_sizes':[(10,5,3), (20,7,3)], 'activation':['tanh', 'relu'], 'learning_rate':['constant', 'adaptive'], 'max_iter' :[100, 150]}
#parameters = {'hidden_layer_sizes':[(10,), (15,), (10,5), (20,7,3)]}

mlp_random = RandomizedSearchCV(mlp,parameters,n_iter=15)
mlp_random.fit(X_train1, y_train)
grid_parm=mlp_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the MLPClassifier 
mlp = MLPClassifier(**grid_parm)
mlp.fit(X_train1,y_train)
mlp_predict = mlp.predict(X_test1)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for MultiLayer Perceptron:{0:6f}".format(mlp.score(X_test1,y_test)))
print("Confusion Matrix after hypertuning for MultiLayer Perceptron")
print(confusion_matrix(y_test,mlp_predict))
print("=== Classification Report ===")
print(classification_report(y_test,mlp_predict))

#get cross-validation report
mlp_cv_score = cross_val_score(mlp, X_train1, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(mlp_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - MultiLayer Perceptron: ",mlp_cv_score.mean())

In [ ]:
#Construct K-Nearest Neighbor Model

neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
neigh_predict=neigh.predict(X_test)
print("accuracy Score (training) for KNeighborsClassifier:{0:6f}".format(neigh.score(X_test,y_test)))
print("Confusion Matrix for KNeighborsClassifier:")
print(confusion_matrix(y_test,neigh_predict))

In [ ]:
#Hyperparameter tuning done for K-Nearest Neighbor classifier

parameters = {'n_neighbors':[3,5,7,9,11], 'weights':['uniform', 'distance'], 'p':[1,2]}


neigh_random = RandomizedSearchCV(neigh,parameters,n_iter=15)
neigh_random.fit(X_train1, y_train)
grid_parm=neigh_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the MLPClassifier 
neigh = KNeighborsClassifier(**grid_parm)
neigh.fit(X_train1,y_train)
neigh_predict = neigh.predict(X_test1)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for KNeighborsClassifier:{0:6f}".format(neigh.score(X_test1,y_test)))
print("Confusion Matrix after hypertuning for KNeighborsClassifier")
print(confusion_matrix(y_test,neigh_predict))
print("=== Classification Report ===")
print(classification_report(y_test,neigh_predict))

#get cross-validation report
neigh_cv_score = cross_val_score(neigh, X_train1, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(neigh_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - KNeighborsClassifier: ",neigh_cv_score.mean())

In [89]:
#Construct Linear Support Vector Machine Model
from sklearn.svm import LinearSVC 
linsvm = LinearSVC(max_iter=300) 
linsvm.fit(X1_train, y1_train) 
linsvm_predict=linsvm.predict(X1_test) 
# print("accuracy Score (training) for Linear SVM Classifier:{0:6f}".format(linsvm.score(X_test,y_test))) 
# print("Confusion Matrix for Linear SVM Classifier:") 
# print(confusion_matrix(y_test,linsvm_predict))

# SMOTE

In [193]:
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy=1)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))
print(X_res.shape)
print(y_res.shape)

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 42133, 1: 9867})
Resampled dataset shape Counter({0: 42133, 1: 42133})
(84266, 595)
(84266,)


In [197]:
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ KNeighborsClassifier(), MLPClassifier(), LinearSVC(), RandomForestClassifier(), DecisionTreeClassifier() ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.71728295]
    fold  1:  [0.71557412]
    fold  2:  [0.72078230]
    fold  3:  [0.71598785]
    ----
    MEAN:     [0.71740681] + [0.00204826]
    FULL:     [0.71740678]

model  1:     [MLPClassifier]
    fold  0:  [0.87046091]
    fold  1:  [0.87648930]
    fold  2:  [0.63818475]
    fold  3:  [0.87909428]
    ----
    MEAN:     [0.81605731] + [0.10274250]
    FULL:     [0.81605867]

model  2:     [LinearSVC]
    fold  0:  [0.57540229]
    fold  1:  [0.61489533]
    fold  2:  [0.56441660]
    fold  3:  [0.57239153]
    ----
    MEAN:     [0.58177644] + [0.01953799]
    FULL:     [0.58177675]

model  3:     [RandomForestCla

In [198]:
model = DecisionTreeClassifier()
    
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_test, y_pred))
y_pred

Final prediction score for ensemble methods: [0.90146154]


array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

In [204]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(1,50,2),'max_depth': 
            range(5,20,1),'criterion':['gini','entropy']}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(S_Train, y_res)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(S_Train, y_res)
clf_predict1 = clf.predict(S_Test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for Decision Tree:{0:6f}".format(clf.score(S_Train, y_res)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict1))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict1))

#get cross-validation report
clf_cv_score = cross_val_score(clf, S_Train, y_res, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

{'min_samples_split': 33, 'max_depth': 7, 'criterion': 'entropy'}
accuracy Score (training) after hypertuning for Decision Tree:0.942195
Confusion Matrix after hypertuning for Decision Tree
[[10399   206]
 [ 1075  1320]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.91      0.98      0.94     10605
           1       0.87      0.55      0.67      2395

    accuracy                           0.90     13000
   macro avg       0.89      0.77      0.81     13000
weighted avg       0.90      0.90      0.89     13000

=== All AUC Scores ===
[0.91423593 0.91703269 0.97036035 0.99651217 0.9961229  0.9949693
 0.99640615 0.9966443  0.99656951 0.997648  ]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.9776501291962225


In [205]:
model = MLPClassifier()
    
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_test, y_pred))
y_pred

Final prediction score for ensemble methods: [0.90138462]


array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

In [208]:
#Hyperparameter tuning done for MultiLayer Perceptron classifier

#parameters = {'hidden_layer_sizes':[(10,), (20,)], 'activation':['tanh', 'relu'], 'solver':['sgd', 'adam'], 'alpha': [0.0001, 0.05], 'learning_rate':['constant', 'adaptive']}
#parameters = {'hidden_layer_sizes':[(10,5), (20,5)], 'activation':['tanh', 'relu'], 'learning_rate':['constant', 'adaptive']}
parameters = {'hidden_layer_sizes':[(10,5,3), (20,7,3)], 'activation':['tanh', 'relu'], 'learning_rate':['constant', 'adaptive'], 'max_iter' :[100, 150]}
#parameters = {'hidden_layer_sizes':[(10,), (15,), (10,5), (20,7,3)]}

mlp_grid = GridSearchCV(mlp,parameters)
mlp_grid.fit(S_Train, y_res)
grid_parm=mlp_grid.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the MLPClassifier 
mlp = MLPClassifier(**grid_parm)
mlp.fit(S_Train, y_res)
mlp_predict = mlp.predict(S_Test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for MultiLayer Perceptron:{0:6f}".format(mlp.score(S_Train, y_res)))
print("Confusion Matrix after hypertuning for MultiLayer Perceptron")
print(confusion_matrix(y_test,mlp_predict))
print("=== Classification Report ===")
print(classification_report(y_test,mlp_predict))

#get cross-validation report
mlp_cv_score = cross_val_score(mlp, S_Train, y_res, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(mlp_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - MultiLayer Perceptron: ",mlp_cv_score.mean())

{'activation': 'tanh', 'hidden_layer_sizes': (10, 5, 3), 'learning_rate': 'constant', 'max_iter': 150}
accuracy Score (training) after hypertuning for MultiLayer Perceptron:0.941791
Confusion Matrix after hypertuning for MultiLayer Perceptron
[[10388   217]
 [ 1074  1321]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.91      0.98      0.94     10605
           1       0.86      0.55      0.67      2395

    accuracy                           0.90     13000
   macro avg       0.88      0.77      0.81     13000
weighted avg       0.90      0.90      0.89     13000

=== All AUC Scores ===
[0.91436075 0.91682236 0.97038063 0.99651437 0.99613286 0.99496845
 0.99637573 0.99660132 0.99637503 0.99768102]


=== Mean AUC Score ===
Mean AUC Score - MultiLayer Perceptron:  0.9776212505758057


In [210]:
model = KNeighborsClassifier()
    
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_test, y_pred))
y_pred

Final prediction score for ensemble methods: [0.90253846]


array([0, 0, 1, ..., 1, 0, 0], dtype=int64)

In [211]:
#Hyperparameter tuning done for K-Nearest Neighbor classifier

parameters = {'n_neighbors':[3,5,7,9,11], 'weights':['uniform', 'distance'], 'p':[1,2]}


neigh_random = RandomizedSearchCV(neigh,parameters,n_iter=15)
neigh_random.fit(S_Train, y_res)
grid_parm=neigh_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the MLPClassifier 
neigh = KNeighborsClassifier(**grid_parm)
neigh.fit(S_Train, y_res)
neigh_predict = neigh.predict(S_Test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for KNeighborsClassifier:{0:6f}".format(neigh.score(S_Train, y_res)))
print("Confusion Matrix after hypertuning for KNeighborsClassifier")
print(confusion_matrix(y_test,neigh_predict))
print("=== Classification Report ===")
print(classification_report(y_test,neigh_predict))

#get cross-validation report
neigh_cv_score = cross_val_score(neigh, S_Train, y_res, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(neigh_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - KNeighborsClassifier: ",neigh_cv_score.mean())

{'weights': 'distance', 'p': 1, 'n_neighbors': 7}
accuracy Score (training) after hypertuning for KNeighborsClassifier:0.938053
Confusion Matrix after hypertuning for KNeighborsClassifier
[[10401   204]
 [ 1059  1336]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.91      0.98      0.94     10605
           1       0.87      0.56      0.68      2395

    accuracy                           0.90     13000
   macro avg       0.89      0.77      0.81     13000
weighted avg       0.90      0.90      0.89     13000

=== All AUC Scores ===
[0.51198679 0.86653724 0.95476071 0.99025214 0.99261794 0.99213663
 0.99359597 0.99292938 0.99251517 0.9932488 ]


=== Mean AUC Score ===
Mean AUC Score - KNeighborsClassifier:  0.9280580772653385


In [213]:
#Hyperparameter tuning for random forest classifier
rand_parameters={'min_samples_leaf' : range(1,10,1),'max_depth': 
            range(5,10,1),'max_features':range(30,60,2),'n_estimators':range(8,28,2)}

rfc_random = RandomizedSearchCV(rfc,rand_parameters,n_iter=15)
rfc_random.fit(S_Train, y_res)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

#Construct Random Forest with best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(S_Train, y_res)
rfc_predict = rfc.predict(S_Test)

print("accuracy Score (training) after hypertuning for Random Forest:{0:6f}".format(rfc.score(S_Train, y_res)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))

#get cross-validation report
rfc_cv_score = cross_val_score(rfc, S_Train, y_res, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())

{'n_estimators': 26, 'min_samples_leaf': 9, 'max_features': 56, 'max_depth': 7}
accuracy Score (training) after hypertuning for Random Forest:0.942183
Confusion Matrix after hypertuning for Random Forest:
[[10399   206]
 [ 1076  1319]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.91      0.98      0.94     10605
           1       0.86      0.55      0.67      2395

    accuracy                           0.90     13000
   macro avg       0.89      0.77      0.81     13000
weighted avg       0.90      0.90      0.89     13000

=== All AUC Scores ===
[0.91430679 0.91703021 0.97036761 0.99651498 0.99612075 0.99496902
 0.99640615 0.99662537 0.99656951 0.997648  ]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.9776558402704543
